# LAB 04 - Data Preprocessing 2

## 1. 열 기준 병합

In [ ]:
# join은 거의 안씀
# -> join ⊂ merge, merge 기능이 join 기능 다 포함함

# merge, join: 옆으로(열) 병합
# concat: 아래로(행) 병합
# key: inner/outer join on의 'on'에 해당하는 것

# DBMS는 동시접속자(같은회사사람) 개 많음 -> 성능에 영향 주는 outer join 쓰기에 적절치 않아서 inner join을 많이 쓰는데
# python은 컴터당 돌아가는 개별 프로그램이여서 내가 원하는 조인문(inner, outer) 상관없이 사용 가능

# ∴ 병합 기준:
# 열: left/right_on: ['컬럼'] 기준
# 행: left/right_index = True: 인덱스(행이름) 기준

### #01. 준비작업

In [3]:
# 1. 패키지 참조:
from hossam import load_data
from pandas import DataFrame
from pandas import merge

In [ ]:
# 2. 샘플 데이터 가져오기

# 1) 첫 번째 샘플 데이터:
고객 = load_data("customer_no_name")
고객

[data] https://data.hossam.kr/data/lab04/customer-no-name.xlsx
[desc] 데이터프레임 병합 실습을 위한 고객번호와 고객 이름에 대한 가상 데이터(인덱스와 메타데이터 없음)
[!] Cannot read metadata


,고객번호,이름
0,1001,둘리
1,1002,도우너
2,1003,또치
3,1004,길동
4,1005,희동
5,1006,마이콜
6,1007,영희


In [7]:
# 2) 두 번째 샘플 데이터:
매출 = load_data("customer_no_pay")
매출

[data] https://data.hossam.kr/data/lab04/customer-no-pay.xlsx
[desc] 데이터프레임 병합 실습을 위한 고객번호와 매출액에 대한 가상 데이터(인덱스와 메타데이터 없음)
[!] Cannot read metadata


,고객번호,금액
0,1001,10000
1,1001,20000
2,1005,15000
3,1006,5000
4,1008,100000
5,1001,30000


### #02. 열 기준 병합

In [ ]:
# 1. 일치하는 데이터끼리 병합 (SQL의 inner join과 동일함):
    # - 두개의 데이터 프레임에서 이름이 동일한 컬럼을 기준으로 
    #   같은 데이터끼리 병합하고 일치하지 않는 데이터는 버려진다.
    # - 만약 양쪽 데이터프레임의 공통 컬럼에 중복데이터가 여러 개 있는 경우는 
    #   모든 경우의 수를 따져서 조합을 만들어낸다. (예: 1001번 둘리의 데이터)
merge(고객, 매출)

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


In [ ]:
# 2. 왼쪽 데이터프레임을 기준으로 병합 (SQL의 LEFT OUTER JOIN과 동일):
    # - 고객(왼쪽) 데이터를 기준으로 일치하는 매출데이터를 병합한다.
    #   -> 고객 데이터 모든 항목 출력 보장
    #   -> 고객 데이터에 매칭되는 매출 데이터가 없는 경우는 빈 값으로 처리된다
merge(고객, 매출, how='left')

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN


In [ ]:
# 3. 오른쪽 데이터프레임을 기준으로 병합 (SQL의 RIGHT OUTER JOIN과 동일):
merge(고객, 매출, how='right')

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1005,희동,15000
3,1006,마이콜,5000
4,1008,NaN,100000
5,1001,둘리,30000


### #03. 병합 기준 열 지정하기

In [ ]:
# case study1: 두 데이터프레임에서 동일 이름의 열 w/ different 데이터 타입

# 1. 샘플 데이터 가져오기:
cd1 = load_data("customer_data1")
cd1

[data] https://data.hossam.kr/data/lab04/customer_data1.xlsx
[desc] 데이터 병합용 실습 데이터 (1) (인덱스/메타데이터 없음)
[!] Cannot read metadata


,고객명,날짜,데이터
0,민수,2018-01-01,20000
1,수영,2018-01-01,100000


In [13]:
cd2 = load_data("customer_data2")
cd2

[data] https://data.hossam.kr/data/lab04/customer_data2.xlsx
[desc] 데이터 병합용 실습 데이터 (2) (인덱스/메타데이터 없음)
[!] Cannot read metadata


,고객명,데이터
0,민수,21세
1,수영,20세


In [ ]:
# 2. 기본 병합:
# - 두 데이터프레임에서 이름이 같은 열은 모두 키가 됨
# - 샘플데이터에서는 '데이터'라는 이름의 변수가 cd1은 int, cd2는 str 타입이므로 
#   병합 기준을 충족하지 않음!
# - 그러므로 아래 코드는 에러임!!

# merge(cd1, cd2)

In [16]:
# 3. 병합기준 설정하기
# - 'on' 파라미터 옵견으로 병합의 기준이 되는 열을 지정해야 한다.
# - 병합 기준 열이 아니면서 이름이 같은 열에는 _x 또는 _y와 같은 
#   prefix가 자동으로 붙게됨:
tmp = merge(cd1, cd2, on = '고객명')
tmp


,고객명,날짜,데이터_x,데이터_y
0,민수,2018-01-01,20000,21세
1,수영,2018-01-01,100000,20세


In [ ]:
# case study2: 두 데이터프레임의 모든 컬럼의 이름이 다를 때

# 해당 케이스의 두 데이터프레임 생성:
df_left = DataFrame(
    {
        '이름': ['영희', '철수'],
        '국어': [87, 91]
    }
)
df_left

,이름,국어
0,영희,87
1,철수,91


In [18]:
df_right = DataFrame(
    {
        '성명': ['영희', '철수'],
        '영어': [90, 82]
    }
)
df_right

,성명,영어
0,영희,90
1,철수,82


In [20]:
# 4. 위 경우에선 왼쪽의 기준열 이름과 오른쪽 기준열 이름을 각각 설정해야 됨:
r3 = merge(df_left, df_right, left_on = ['이름'], right_on = ['성명'])
r3
    # 기준으로 지정된 모든 변수가 포함되므로 
    # 불필요한 변수는 drop() 함수를 통해 삭제할 필요가 있다.

,이름,국어,성명,영어
0,영희,87,영희,90
1,철수,91,철수,82


### #04. case study3: 인덱스를 활용한 병합방법

In [ ]:
# 1. 학생의 이름을 인덱스로 갖는 두 데이터 프레임 생성:
df_left = DataFrame(
    {'수학': [90, 82]},
    index = ['민철', '봉구']
)
df_left

,수학
민철,90
봉구,82


In [23]:
df_right = DataFrame(
    {'국어': [90, 82]},
    index = ['민철', '철수']
)
df_right

,국어
민철,90
철수,82


In [24]:
# 2. 병합 조건에서 index를 사용하도록 지정하여 Inner Join하기:
merge(
    df_left,
    df_right,
    left_index = True, 
    right_index = True
)

,수학,국어
민철,90,90


In [25]:
# 3. 병합 조건에서 index를 사용하도록 지정하여 Left Outer Join하기:
merge(
    df_left,
    df_right,
    left_index = True, 
    right_index = True,
    how = 'left'
)

,수학,국어
민철,90,90.0
봉구,82,NaN


In [26]:
# 4. 병합 조건에서 index를 사용하도록 지정하여 Right Outer Join하기:
merge(
    df_left,
    df_right,
    left_index = True, 
    right_index = True,
    how = 'right'
)

,수학,국어
민철,90.0,90
철수,NaN,82


In [27]:
# 5. 병합 조건에서 index를 사용하도록 지정하여 Full Outer Join하기:
merge(
    df_left,
    df_right,
    left_index = True, 
    right_index = True,
    how = 'outer'
)

,수학,국어
민철,90.0,90.0
봉구,82.0,NaN
철수,NaN,82.0


### #05. case study4: 인덱스와 컬럼을 각각 기준으로 할 때

In [ ]:
# 1. 해당 케이스의 샘플 데이터프레임 생성:

# - 이름이 인덱스인 데이터프레임:
df_left = DataFrame(
    {'수학': [90, 82]},
    index = ['민철', '봉구']
)
df_left

,수학
민철,90
봉구,82


In [30]:
# - 이름이 변수(컬럼)에 있는 데이터프레임:
df_right = DataFrame(
    {
        '성명': ['민철', '철수'],
        '영어': [90, 82]
    }
)
df_right

,성명,영어
0,민철,90
1,철수,82


In [34]:
# 2. 왼쪽은 인덱스, 오른쪽은 컬럼을 기준으로 Inner Join 하는 법:
merge(
    df_left,
    df_right,
    left_index = True, 
    right_on = ['성명']
)

,수학,성명,영어
0,90,민철,90


In [35]:
# 3. 왼쪽은 인덱스, 오른쪽은 컬럼을 기준으로 Left Outer Join 하는 법:
merge(
    df_left,
    df_right,
    left_index = True, 
    right_on = ['성명'],
    how = 'left'
)

,수학,성명,영어
0.0,90,민철,90.0
NaN,82,봉구,NaN


In [36]:
# 4. 왼쪽은 인덱스, 오른쪽은 컬럼을 기준으로 Right Outer Join 하는 법:
merge(
    df_left,
    df_right,
    left_index = True, 
    right_on = ['성명'],
    how = 'right'
)

,수학,성명,영어
0,90.0,민철,90
1,NaN,철수,82


In [37]:
# 5. 왼쪽은 인덱스, 오른쪽은 컬럼을 기준으로 Full Outer Join 하는 법:
tmp = merge(
    df_left,
    df_right,
    left_index = True, 
    right_on = ['성명'],
    how = 'outer'
)
tmp

,수학,성명,영어
0.0,90.0,민철,90.0
NaN,82.0,봉구,NaN
1.0,NaN,철수,82.0


In [38]:
# 6. 병합 후엔 인덱스 정리해야됨:
tmp2 = tmp.set_index('성명')
tmp2

,수학,영어
성명,,
민철,90.0,90.0
봉구,82.0,NaN
철수,NaN,82.0
